# Setup 

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
import pickle
import subprocess

In [10]:
project_dir = Path('/data/nielsond/meld/')
%pwd
%cd {project_dir}
%pwd

/spin1/users/nielsond/meld


'/spin1/users/nielsond/meld'

In [3]:
bids_dir = Path('data/voice/')
aprocdir = Path('data/voice/derivatives/afni_proc2')
nbdir = Path('notebooks')

outdir = Path('data/voice/derivatives/meld')
stim_dir = bids_dir/'afni_stim'
#!rm -rf {outdir}
singularity_image = Path('singularity/meldlocal-2017-12-13-11dcc4d4be31.img')
swarm_path = Path('proc_swarm_meld_voice.cmd')
log_dir = Path('swarm_log')

## Create commands for replicabilty experiment

In [4]:
bad_list = ['sub-007', 'sub-023', 'sub-072', 'sub-077', 'sub-137', 'sub-165', 'sub-045']

In [5]:
#tmp_df = pd.read_csv('../data/voice/task-passivelistening_events.tsv', sep = '\t')
#tmp_df.stimuli = tmp_df.stimuli.str.split('/').str[-1]


In [6]:
stim_perm_file = Path('stim_perms.pklz')

In [11]:
try:
    with stim_perm_file.open('rb') as h:
        n_stims = pickle.load(h)
except FileNotFoundError:
    raise Exception("no stim file found")
    n_stims = list([(ii, sorted(np.random.choice(range(1,21), size = 10, replace = False))) for ii in range(2,102)])
    with stim_perm_file.open('wb') as h:
        pickle.dump(n_stims, h)

In [51]:
def make_sing_call(task, batch_name, run_n, aprocdir, nbdir, outdir=None, 
                   singularity_image=None, subject=None, stimuli=None,
                   no_tfce=False, no_downsample=False, no_item=False,
                   openblas_threads=5):

    lscratch = '/lscratch/$SLURM_JOBID/'
    derivative_dir = lscratch+'meld/data/%s/derivatives'%task
    bic = '/work/' + bids_dir.parts[-1]
    oic = '/meld_root/work/data/%s/derivatives/meld'%task
    cmd = 'export OPENBLAS_NUM_THREADS=%d; mkdir -p '%openblas_threads + derivative_dir + \
    '; rsync -avch --include="*/" --include="*.nii.gz" --exclude="*" ' + aprocdir.absolute().as_posix() + ' ' +derivative_dir +'/'+ \
    '; rsync -avch ' + nbdir.absolute().as_posix() + ' ' + lscratch + 'meld/' + \
    '; rsync -avch /data/nielsond/meld/data/voice/task-passivelistening_events.tsv ' + lscratch + 'meld/data/voice/' + \
    '; singularity exec' + \
    ' -B ' + lscratch +'meld/data:/meld_root/work/data' + \
    ' -B ' + lscratch +'meld/notebooks:/meld_root/work/code' + \
    ' ' + singularity_image.absolute().as_posix()  + \
    ' python3 /meld_root/work/code/run_%s_meld.py '%task + oic + ' ' + str(batch_name)+ ' ' + str(run_n)
    if stimuli is not None:
        cmd += ' --stimuli="%s"'%stimuli
    if subject is not None:
        cmd += ' --subjects="%s"'%subject
    if no_tfce:
        cmd += ' --no_tfce'
    if no_downsample:
        cmd += ' --no_downsample'
    if no_item:
        cmd += ' --no_item'
    cmd += '; rsync -avch {derivative_dir}/meld/ '.format(derivative_dir = derivative_dir) + outdir.absolute().as_posix() + '/ ;'  
    cmd += ' rm -rf /lscratch/$SLURM_JOB_ID/*'
    return cmd

def run_cmd(cmd):
    output = !{cmd}
    return output

In [26]:
cmds = []
for run_n, stim_ns in n_stims:
    stimuli = ['vocal_%02d'%sn for sn in stim_ns] +['non_vocal_%02d'%sn for sn in stim_ns]
    cmds.append(make_sing_call('voice','repl-nods',run_n,aprocdir,nbdir,outdir,singularity_image, stimuli=' '.join(stimuli), no_downsample = True))

In [27]:
swarm_path.write_text('\n'.join(cmds[2:]))
                      
print(swarm_path.read_text())

export OPENBLAS_NUM_THREADS=5; mkdir -p /lscratch/$SLURM_JOBID/meld/data/voice/derivatives; rsync -avch --include="*/" --include="*.nii.gz" --exclude="*" /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2 /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/; rsync -avch /spin1/users/nielsond/meld/notebooks /lscratch/$SLURM_JOBID/meld/; rsync -avch /data/nielsond/meld/data/voice/task-passivelistening_events.tsv /lscratch/$SLURM_JOBID/meld/data/voice/; singularity exec -B /lscratch/$SLURM_JOBID/meld/data:/meld_root/work/data -B /lscratch/$SLURM_JOBID/meld/notebooks:/meld_root/work/code /spin1/users/nielsond/meld/singularity/meldlocal-2017-12-13-11dcc4d4be31.img python3 /meld_root/work/code/run_voice_meld.py /meld_root/work/data/voice/derivatives/meld repl-nods 4 --stimuli="vocal_01 vocal_04 vocal_06 vocal_07 vocal_09 vocal_10 vocal_11 vocal_14 vocal_18 vocal_19 non_vocal_01 non_vocal_04 non_vocal_06 non_vocal_07 non_vocal_09 non_vocal_10 non_vocal_11 non_vocal_14 non_vocal_18 no

In [28]:
if not log_dir.exists():
    log_dir.mkdir()

!swarm -f {swarm_path} --module singularity -g 100 -t 56 --logdir {log_dir} --time 36:00:00 --partition=nimh,norm --gres=lscratch:200

56575132


In [29]:
repl_paths = sorted(outdir.glob('*repl*_t_thresh.nii.gz'))
complete_runs = [int(rp.parts[-1].split('_')[2]) for rp in repl_paths]
failed_runs = [rn for rn in range(2,102) if rn not in complete_runs]

ValueError: invalid literal for int() with base 10: 'nods-noi'

In [285]:
cmds = []
for run_n, stim_ns in n_stims:
    if run_n in failed_runs:
        stimuli = ['vocal_%02d'%sn for sn in stim_ns] +['non_vocal_%02d'%sn for sn in stim_ns]
        cmds.append(make_sing_call('voice','repl',run_n,aprocdir,nbdir,outdir,singularity_image, stimuli=' '.join(stimuli)))

In [287]:
swarm_path.write_text('\n'.join(cmds[:]))
                      
print(swarm_path.read_text())

export OPENBLAS_NUM_THREADS=5; mkdir -p /lscratch/$SLURM_JOBID/meld/data/voice/derivatives; rsync -avch --include="*/" --include="*.nii.gz" --exclude="*" /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2 /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/; rsync -avch /spin1/users/nielsond/meld/notebooks /lscratch/$SLURM_JOBID/meld/; rsync -avch /data/nielsond/meld/data/voice/task-passivelistening_events.tsv /lscratch/$SLURM_JOBID/meld/data/voice/; singularity exec -B /lscratch/$SLURM_JOBID/meld/data:/meld_root/work/data -B /lscratch/$SLURM_JOBID/meld/notebooks:/meld_root/work/code /spin1/users/nielsond/meld/singularity/meldlocal-2017-12-13-11dcc4d4be31.img python3 /meld_root/work/code/run_voice_meld.py /meld_root/work/data/voice/derivatives/meld repl 9 --stimuli="vocal_01 vocal_06 vocal_07 vocal_08 vocal_09 vocal_12 vocal_13 vocal_17 vocal_18 vocal_20 non_vocal_01 non_vocal_06 non_vocal_07 non_vocal_08 non_vocal_09 non_vocal_12 non_vocal_13 non_vocal_17 non_vocal_18 non_voc

In [288]:
!swarm -f {swarm_path} --module singularity -g 100 -t 56 --logdir {log_dir} --time 36:00:00 --partition=nimh,norm --gres=lscratch:200

56512466


In [30]:
cmds = []
for run_n, stim_ns in n_stims:
    stimuli = ['vocal_%02d'%sn for sn in stim_ns] +['non_vocal_%02d'%sn for sn in stim_ns]
    cmds.append(make_sing_call('voice','repl-nods-noi',run_n,aprocdir,nbdir,outdir,
                               singularity_image, stimuli=' '.join(stimuli),
                               no_downsample = True, no_item = True))

In [31]:
swarm_path.write_text('\n'.join(cmds[:]))
                      
print(swarm_path.read_text())
if not log_dir.exists():
    log_dir.mkdir()



export OPENBLAS_NUM_THREADS=5; mkdir -p /lscratch/$SLURM_JOBID/meld/data/voice/derivatives; rsync -avch --include="*/" --include="*.nii.gz" --exclude="*" /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2 /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/; rsync -avch /spin1/users/nielsond/meld/notebooks /lscratch/$SLURM_JOBID/meld/; rsync -avch /data/nielsond/meld/data/voice/task-passivelistening_events.tsv /lscratch/$SLURM_JOBID/meld/data/voice/; singularity exec -B /lscratch/$SLURM_JOBID/meld/data:/meld_root/work/data -B /lscratch/$SLURM_JOBID/meld/notebooks:/meld_root/work/code /spin1/users/nielsond/meld/singularity/meldlocal-2017-12-13-11dcc4d4be31.img python3 /meld_root/work/code/run_voice_meld.py /meld_root/work/data/voice/derivatives/meld repl-nods-noi 2 --stimuli="vocal_01 vocal_02 vocal_03 vocal_04 vocal_07 vocal_12 vocal_13 vocal_15 vocal_16 vocal_18 non_vocal_01 non_vocal_02 non_vocal_03 non_vocal_04 non_vocal_07 non_vocal_12 non_vocal_13 non_vocal_15 non_vocal_1

In [32]:
!swarm -f {swarm_path} --module singularity -g 100 -t 56 --logdir {log_dir} --time 36:00:00 --partition=nimh,norm --gres=lscratch:200

56575133


In [52]:
# Run full model with and without item level random effect
cmds = []
cmds.append(make_sing_call('voice','repl-nods-noi',0,aprocdir,nbdir,outdir,
                           singularity_image,
                            no_downsample = True, no_item = True))
cmds.append(make_sing_call('voice','repl-nods',0,aprocdir,nbdir,outdir,
                           singularity_image,
                            no_downsample = True, no_item = True))

swarm_path.write_text('\n'.join(cmds[:]))
                      
print(swarm_path.read_text())
if not log_dir.exists():
    log_dir.mkdir()


export OPENBLAS_NUM_THREADS=5; mkdir -p /lscratch/$SLURM_JOBID/meld/data/voice/derivatives; rsync -avch --include="*/" --include="*.nii.gz" --exclude="*" /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2 /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/; rsync -avch /spin1/users/nielsond/meld/notebooks /lscratch/$SLURM_JOBID/meld/; rsync -avch /data/nielsond/meld/data/voice/task-passivelistening_events.tsv /lscratch/$SLURM_JOBID/meld/data/voice/; singularity exec -B /lscratch/$SLURM_JOBID/meld/data:/meld_root/work/data -B /lscratch/$SLURM_JOBID/meld/notebooks:/meld_root/work/code /spin1/users/nielsond/meld/singularity/meldlocal-2017-12-13-11dcc4d4be31.img python3 /meld_root/work/code/run_voice_meld.py /meld_root/work/data/voice/derivatives/meld repl-nods-noi 0 --no_downsample --no_item; rsync -avch /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/meld/ /spin1/users/nielsond/meld/data/voice/derivatives/meld/ ; rm -rf /lscratch/$SLURM_JOB_ID/*
export OPENBLAS_NUM_THREADS=5

In [53]:
!swarm -f {swarm_path} --module singularity -g 100 -t 56 --logdir {log_dir} --time 36:00:00 --partition=nimh,norm --gres=lscratch:200

56608625


# Make FSL event files for replicability

In [154]:
tr = 2
num_points = 310
trs_dropped = 3
tmp_df['stim_num'] = (tmp_df.stimuli.str.split('_').str[-1]).astype(int)
tmp_df['adj_onset'] = tmp_df.onset - (tr * trs_dropped)
tmp_df['tr_onset'] = (tmp_df.adj_onset / tr).astype(int)
tmp_df['tr_duration'] = (tmp_df.duration / tr).astype(int)
tmp_df['tr_end']= tmp_df.tr_onset+tmp_df.tr_duration

In [155]:
tmp_df

,onset,duration,trial_type,stimuli,stim_num,adj_onset,tr_onset,tr_duration,tr_end
0,12,8,nonvocal,non_vocal_01,01,6,3,4,7
1,22,8,vocal,vocal_01,01,16,8,4,12
2,32,8,nonvocal,non_vocal_02,02,26,13,4,17
3,52,8,nonvocal,non_vocal_03,03,46,23,4,27
4,62,8,vocal,vocal_02,02,56,28,4,32
5,82,8,vocal,vocal_03,03,76,38,4,42
6,102,8,nonvocal,non_vocal_04,04,96,48,4,52
7,112,8,vocal,vocal_04,04,106,53,4,57
8,122,8,nonvocal,non_vocal_05,05,116,58,4,62
9,132,8,vocal,vocal_05,05,126,63,4,67


In [190]:
fsl_design_outdir = Path('fsl_design_mats')
if not fsl_design_outdir.exists():
    fsl_design_outdir.mkdir()

on_val = '7.419355e-01'
off_val = '-2.580645e-01'

#write first mat with everything
run_n = 0
tcs = []
for tt in ['vocal', 'nonvocal']:
    tc = [off_val] *num_points
    for ii,row in tmp_df.query("trial_type == @tt").iterrows():
        tc[row.tr_onset:row.tr_end] = [on_val]*row.tr_duration
    tcs.append(tc)

    # make design.mat header
    design_txt = []
    design_txt.append('/NumWaves\t%d'%len(tmp_df.trial_type.unique()))
    design_txt.append('/NumPoints\t%d'%num_points)
    design_txt.append('/PPheights \t1.000000e+00\t1.000000e+00')
    design_txt.append('')
    design_txt.append('/Matrix')
    design_txt.extend(['\t'.join(ii) for ii in list(zip(*tcs))])

    fsl_out = fsl_design_outdir/('design_%03d.mat'%run_n)
    fsl_out.write_text('\n'.join(design_txt))

# write the rest of the mats
for run_n, stim_ns in n_stims:
    tcs = []
    for tt in ['vocal', 'nonvocal']:
        tc = [off_val] *num_points
        for ii,row in tmp_df.query("trial_type == @tt and stim_num in @stim_ns").iterrows():
            tc[row.tr_onset:row.tr_end] = [on_val]*row.tr_duration
        tcs.append(tc)
        
        # make design.mat header
        design_txt = []
        design_txt.append('/NumWaves\t%d'%len(tmp_df.trial_type.unique()))
        design_txt.append('/NumPoints\t%d'%num_points)
        design_txt.append('/PPheights \t1.000000e+00\t1.000000e+00')
        design_txt.append('')
        design_txt.append('/Matrix')
        design_txt.extend(['\t'.join(ii) for ii in list(zip(*tcs))])

        fsl_out = fsl_design_outdir/('design_%03d.mat'%run_n)
        fsl_out.write_text('\n'.join(design_txt))

# Run AFNI for replicability
### First level analysis

In [346]:
#make afni timing file
for run_n, stim_ns in n_stims:
    for tt in np.unique(bids_stims.trial_type):
        stim_file = stim_dir/('task-passivelistening_%s_%03d_stim-%s.txt'%('repl',run_n,tt))
        stim_file.write_text(' '.join(list((tmp_df.query('trial_type == @tt and stim_num in @ stim_ns').onset.values).astype('str'))))

In [541]:
cmd = """module load afni; cd {proc_dir};
3dDeconvolve -input pb04.{subj}.r*.scale+tlrc.HEAD                        
    -censor motion_{subj}_censor.1D                                     
    -polort 5 -float                                                     
    -num_stimts 8                                                        
    -stim_times 1 {stim_dir}/task-passivelistening_repl_{run_n}_stim-nonvocal.txt     
    'BLOCK(8,1)'                                                         
    -stim_label 1 nonvocal                                               
    -stim_times 2 {stim_dir}/task-passivelistening_repl_{run_n}_stim-vocal.txt        
    'BLOCK(8,1)'                                                        
    -stim_label 2 vocal                                                  
    -stim_file 3 mot_demean.r01.1D'[0]' -stim_base 3 -stim_label 3 roll 
    -stim_file 4 mot_demean.r01.1D'[1]' -stim_base 4 -stim_label 4 pitch 
    -stim_file 5 mot_demean.r01.1D'[2]' -stim_base 5 -stim_label 5 yaw   
    -stim_file 6 mot_demean.r01.1D'[3]' -stim_base 6 -stim_label 6 dS    
    -stim_file 7 mot_demean.r01.1D'[4]' -stim_base 7 -stim_label 7 dL    
    -stim_file 8 mot_demean.r01.1D'[5]' -stim_base 8 -stim_label 8 dP
    -gltsym 'SYM: vocal -nonvocal'
    -glt_label 1 contrast
    -x1D X.xmat.{run_n}.{subj}.1D -nobucket -overwrite;

3dREMLfit -overwrite -matrix X.xmat.{run_n}.{subj}.1D -input pb04.{subj}.r01.scale+tlrc.HEAD \
 -fout -tout -Rbuck stats.repl_{run_n}.{subj}_REML -Rvar stats.repl_{run_n}.{subj}_REMLvar \
 -verb -dsort Local_WMe_rall+tlrc;"""


In [399]:
stim_dir.absolute().as_posix()

'/spin1/users/nielsond/meld/data/voice/afni_stim'

In [480]:
cmds = []
proc_dir = Path('data/voice/derivatives/afni_proc2')
bad_list = ['sub-001','sub-007', 'sub-023', 'sub-072', 'sub-077', 'sub-137', 'sub-165', 'sub-045']
for run_n, stim_ns in n_stims:
    for sub_dir in sorted(proc_dir.glob('sub-*')):
        sub = sub_dir.parts[-1]
        if sub not in bad_list:
            subn = sub_dir.parts[-1][4:]
            scmd = cmd.format(subj = subn,
                       proc_dir = (sub_dir/'passivelistening').absolute().as_posix(),
                       run_n = '%03d'%run_n,
                       stim_dir = stim_dir.absolute().as_posix())
            scmd = ' '.join([cc.strip() for cc in scmd.split('\n')])
            cmds.append(scmd)


In [487]:
240/21

11.428571428571429

In [485]:
ar_swarm_path = Path('proc_swarm_afni_repl_voice.cmd')
ar_swarm_path.write_text('\n'.join(cmds[2:]))


25078799

In [488]:
!swarm -f {ar_swarm_path} --module singularity -g 1 -t 16 -b 21 --logdir {log_dir} --time 00:10:00 --partition=quick,nimh,norm

56550550


In [546]:
cmds = []
# add command to rerun original contrast
for sub_dir in sorted(proc_dir.glob('sub-*')):
    if sub not in bad_list:
        subn = sub_dir.parts[-1][4:]
        scmd = cmd.format(subj = subn,
                   proc_dir = (sub_dir/'passivelistening').absolute().as_posix(),
                   run_n = '%03d'%0,
                   stim_dir = stim_dir.absolute().as_posix())
        scmd = ' '.join([cc.strip() for cc in scmd.split('\n')])
        cmds.append(scmd)
ar_swarm_path = Path('proc_swarm_afni_repl_voice.cmd')
ar_swarm_path.write_text('\n'.join(cmds[:]))

260399

In [545]:
cmds

["module load afni; cd /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2/sub-001/passivelistening; 3dDeconvolve -input pb04.001.r*.scale+tlrc.HEAD -censor motion_001_censor.1D -polort 5 -float -num_stimts 8 -stim_times 1 /spin1/users/nielsond/meld/data/voice/afni_stim/task-passivelistening_repl_000_stim-nonvocal.txt 'BLOCK(8,1)' -stim_label 1 nonvocal -stim_times 2 /spin1/users/nielsond/meld/data/voice/afni_stim/task-passivelistening_repl_000_stim-vocal.txt 'BLOCK(8,1)' -stim_label 2 vocal -stim_file 3 mot_demean.r01.1D'[0]' -stim_base 3 -stim_label 3 roll -stim_file 4 mot_demean.r01.1D'[1]' -stim_base 4 -stim_label 4 pitch -stim_file 5 mot_demean.r01.1D'[2]' -stim_base 5 -stim_label 5 yaw -stim_file 6 mot_demean.r01.1D'[3]' -stim_base 6 -stim_label 6 dS -stim_file 7 mot_demean.r01.1D'[4]' -stim_base 7 -stim_label 7 dL -stim_file 8 mot_demean.r01.1D'[5]' -stim_base 8 -stim_label 8 dP -gltsym 'SYM: vocal -nonvocal' -glt_label 1 contrast -x1D X.xmat.000.001.1D -nobucket -overw

In [547]:
!swarm -f {ar_swarm_path} --module singularity -g 1 -t 16 --logdir {log_dir} --time 00:10:00 --partition=quick,nimh,norm

56552772


### Second level analysis with 3dMEMA

In [620]:
cmd = """module load afni; module load R; python2 /usr/local/apps/afni/current/linux_openmp_64/gen_group_command.py -command 3dMEMA -write_script {proc_dir}/{run_n}.mema_script
-dsets {dsets}
-set_labels VminusNV
-subs_betas '7'
-subs_tstats '8'
-options -jobs 30  -prefix {proc_dir}/{run_n}.mema_result;
cd {proc_dir};
tcsh {proc_dir}/{run_n}.mema_script;
"""

In [621]:
cmds = []
run_n = 0
dsets = proc_dir/'sub-???'/'passivelistening'/('stats.repl_%03d.???_REML+tlrc.HEAD'%run_n)
scmd = cmd.format(dsets = dsets.absolute(), proc_dir = proc_dir.absolute(), run_n = run_n)
scmd = ' '.join([cc.strip() for cc in scmd.split('\n')])
cmds.append(scmd)
for run_n, stim_ns in n_stims:
    dsets = proc_dir/'sub-???'/'passivelistening'/('stats.repl_%03d.???_REML+tlrc.HEAD'%run_n)
    scmd = cmd.format(dsets = dsets.absolute(), proc_dir = proc_dir.absolute(), run_n = "%03d"%run_n)
    scmd = ' '.join([cc.strip() for cc in scmd.split('\n')])
    cmds.append(scmd)


In [622]:
ar_swarm_path = Path('proc_swarm_afni_repl_voice.cmd')
ar_swarm_path.write_text('\n'.join(cmds[:]))

65542

In [618]:
ar_swarm_path.read_text()

"module load afni; module load R; python2 /usr/local/apps/afni/current/linux_openmp_64/gen_group_command.py -command 3dMEMA -write_script /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2/0.mema_script -dsets /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2/sub-???/passivelistening/stats.repl_000.???_REML+tlrc.HEAD -set_labels VminusNV -subs_betas '7' -subs_tstats '8' -options -jobs 30  -prefix /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2/0.mema_result; cd /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2; tcsh /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2/0.mema_script; \nmodule load afni; module load R; python2 /usr/local/apps/afni/current/linux_openmp_64/gen_group_command.py -command 3dMEMA -write_script /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2/002.mema_script -dsets /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2/sub-???/passivelistening/stats.repl_002.???_REML+tlrc.HEAD -set_labels Vm

In [623]:
!swarm -f {ar_swarm_path} --module singularity -g 16 -t 30 --logdir {log_dir} --time 03:59:00 --partition=quick,nimh,norm

56553188


### get blur estimates for every subject

In [ ]:
mask_path = ''
cmd = f'3dFWHMx -mask {sub_dir} -input errts.{subn}_REML+tlrc -acf {sub_dir}/blur_est.{subn}.1D -overwrite >acf_output'

In [633]:
cmds = []
for sub_dir in sorted(proc_dir.glob('sub-*')):
    sub = sub_dir.parts[-1]
    if sub not in bad_list:
        subn = sub_dir.parts[-1][4:]
        cmd = f'module load afni; 3dFWHMx -mask {sub_dir.absolute()}/passivelistening/full_mask.{subn}+tlrc -input {sub_dir.absolute()}/passivelistening/errts.{subn}_REML+tlrc -acf {sub_dir.absolute()}/passivelistening/blur_est.{subn}.1D -overwrite > {sub_dir.absolute()}/passivelistening/acf_input.txt'
        cmds.append(cmd)
ar_swarm_path = Path('proc_swarm_afni_repl_voice.cmd')
ar_swarm_path.write_text('\n'.join(cmds[2:]))

97289

In [634]:
!swarm -f {ar_swarm_path} --module singularity -g 8 -t 8 --logdir {log_dir} --time 00:10:00 --partition=quick,nimh,norm

56561988


# Make singularity comands for power experiment

In [33]:
subj_perm_file = Path('subj_perms.pklz')

In [39]:
proc_dir = Path('data/voice/derivatives/afni_proc2')
bad_list = ['sub-007', 'sub-023', 'sub-072', 'sub-077', 'sub-137', 'sub-165', 'sub-045']
dfs = []
for sub_dir in sorted(proc_dir.glob('sub-*')):
    sub = sub_dir.parts[-1]
    if sub not in bad_list:
        subn = sub_dir.parts[-1][4:]
        stb_dir = sub_dir/'passivelistening/stb/'
        tmp_df = pd.read_csv('data/voice/task-passivelistening_events.tsv', sep = '\t')
        tmp_df.stimuli = tmp_df.stimuli.str.split('/').str[-1]
        tmp_df['subject'] = sub
        dfs.append(tmp_df)
meta = pd.concat(dfs)

In [40]:
# 100 runs each at 5, 10, 20, 50, 100 subjects

subj_perm_file = Path('subj_perms.pklz')
with subj_perm_file.open('rb') as h:
    n_subjs = pickle.load(h)
#try:
#    with subj_perm_file.open('rb') as h:
#        n_subjs = pickle.load(h)
#except FileNotFoundError:
#    subjs_runs = [5, 10, 20, 50, 100]
#    n_subjs = []
#    for ns in subjs_runs:
#        n_subjs.extend(list([('power-subs_%03d'%ns, ii, sorted(np.random.choice(meta.subject.unique(), size = ns, replace = False))) for ii in range(100)]))
#    with subj_perm_file.open('wb') as h:
#        pickle.dump(n_subjs, h)

In [47]:
pwr_cmds = []
for batch_name, run_n, subjs in n_subjs:
    batch_name = batch_name.split('-')[0] + '-nods-nsubs-'+batch_name.split('-')[1].split('_')[1]
    pwr_cmds.append(make_sing_call('voice',batch_name,run_n,
                                   aprocdir,nbdir,outdir,singularity_image, subject=' '.join(subjs)
                                  ,no_downsample=True))

In [48]:
pwr_swarm_path = Path('proc_pwr_swarm_meld_voice.cmd')
pwr_swarm_path.write_text('\n'.join(pwr_cmds[:]))
                      
print(pwr_swarm_path.read_text())

export OPENBLAS_NUM_THREADS=5; mkdir -p /lscratch/$SLURM_JOBID/meld/data/voice/derivatives; rsync -avch --include="*/" --include="*.nii.gz" --exclude="*" /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2 /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/; rsync -avch /spin1/users/nielsond/meld/notebooks /lscratch/$SLURM_JOBID/meld/; rsync -avch /data/nielsond/meld/data/voice/task-passivelistening_events.tsv /lscratch/$SLURM_JOBID/meld/data/voice/; singularity exec -B /lscratch/$SLURM_JOBID/meld/data:/meld_root/work/data -B /lscratch/$SLURM_JOBID/meld/notebooks:/meld_root/work/code /spin1/users/nielsond/meld/singularity/meldlocal-2017-12-13-11dcc4d4be31.img python3 /meld_root/work/code/run_voice_meld.py /meld_root/work/data/voice/derivatives/meld power-nods-nsubs-005 0 --subjects="sub-014 sub-085 sub-147 sub-150 sub-189"; rsync -avch /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/meld/ /spin1/users/nielsond/meld/data/voice/derivatives/meld/ ; rm -rf /lscratch/$SLURM_JOB_

In [49]:
if not log_dir.exists():
    log_dir.mkdir()

!swarm -f {pwr_swarm_path} --module singularity -g 100 -t 56 --logdir {log_dir} --time 36:00:00 --partition=nimh,norm --gres=lscratch:200

56575982


In [ ]:
pwr_cmds = []
for batch_name, run_n, subjs in n_subjs:
    batch_name = batch_name.split('-')[0] + '-nods-noi-nsubs-'+batch_name.split('-')[1].split('_')[1]
    pwr_cmds.append(make_sing_call('voice',batch_name,run_n,
                                   aprocdir,nbdir,outdir,singularity_image, subject=' '.join(subjs)
                                  ,no_downsample=True, no_item = True))

### Deal with failed jobs

In [420]:
pwr_paths = sorted(outdir.glob('*power*noi*_t_thresh.nii.gz'))
complete_runs = [('_'.join([rp.parts[-1].split('_')[1],rp.parts[-1].split('_')[2]]),int(rp.parts[-1].split('_')[3])) for rp in pwr_paths]
failed_runs = [(batch_name, run_n, subjs) for batch_name, run_n, subjs in n_subjs if (batch_name+'noi', run_n) not in complete_runs ]

In [426]:
pwr_cmds = []
for batch_name, run_n, subjs in failed_runs:
    pwr_cmds.append(make_sing_call('voice',batch_name+'noi',run_n,aprocdir,nbdir,outdir,singularity_image, subject=' '.join(subjs), no_item = True))

In [427]:
pwr_swarm_path = Path('proc_pwr_swarm_meld_voice.cmd')
pwr_swarm_path.write_text('\n'.join(pwr_cmds[:]))
                      
print(pwr_swarm_path.read_text())

export OPENBLAS_NUM_THREADS=5; mkdir -p /lscratch/$SLURM_JOBID/meld/data/voice/derivatives; rsync -avch --include="*/" --include="*.nii.gz" --exclude="*" /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2 /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/; rsync -avch /spin1/users/nielsond/meld/notebooks /lscratch/$SLURM_JOBID/meld/; rsync -avch /data/nielsond/meld/data/voice/task-passivelistening_events.tsv /lscratch/$SLURM_JOBID/meld/data/voice/; singularity exec -B /lscratch/$SLURM_JOBID/meld/data:/meld_root/work/data -B /lscratch/$SLURM_JOBID/meld/notebooks:/meld_root/work/code /spin1/users/nielsond/meld/singularity/meldlocal-2017-12-13-11dcc4d4be31.img python3 /meld_root/work/code/run_voice_meld.py /meld_root/work/data/voice/derivatives/meld power-subs_050noi 83 --subjects="sub-002 sub-021 sub-028 sub-034 sub-035 sub-037 sub-038 sub-039 sub-047 sub-050 sub-052 sub-058 sub-060 sub-064 sub-069 sub-071 sub-073 sub-075 sub-076 sub-085 sub-086 sub-087 sub-092 sub-093 sub-098

In [428]:
!swarm -f {pwr_swarm_path} --module singularity -g 100 -t 56 --logdir {log_dir} --time 48:00:00 --partition=nimh,norm --gres=lscratch:200

56547108




for erb in bids_dir.glob('sub-*/func/*emotionalregulation_events.tsv'):
    print(erb)

In [328]:
pwr_cmds = []
for batch_name, run_n, subjs in n_subjs:
    pwr_cmds.append(make_sing_call('voice',batch_name + 'noi',run_n,aprocdir,nbdir,outdir,singularity_image, subject=' '.join(subjs), no_item = True))

In [329]:
pwr_swarm_path = Path('proc_pwr_swarm_meld_voice.cmd')
pwr_swarm_path.write_text('\n'.join(pwr_cmds[:]))
                      
print(pwr_swarm_path.read_text())

export OPENBLAS_NUM_THREADS=5; mkdir -p /lscratch/$SLURM_JOBID/meld/data/voice/derivatives; rsync -avch --include="*/" --include="*.nii.gz" --exclude="*" /spin1/users/nielsond/meld/data/voice/derivatives/afni_proc2 /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/; rsync -avch /spin1/users/nielsond/meld/notebooks /lscratch/$SLURM_JOBID/meld/; rsync -avch /data/nielsond/meld/data/voice/task-passivelistening_events.tsv /lscratch/$SLURM_JOBID/meld/data/voice/; singularity exec -B /lscratch/$SLURM_JOBID/meld/data:/meld_root/work/data -B /lscratch/$SLURM_JOBID/meld/notebooks:/meld_root/work/code /spin1/users/nielsond/meld/singularity/meldlocal-2017-12-13-11dcc4d4be31.img python3 /meld_root/work/code/run_voice_meld.py /meld_root/work/data/voice/derivatives/meld power-subs_005noi 0 --subjects="sub-014 sub-085 sub-147 sub-150 sub-189"--no_item; rsync -avch /lscratch/$SLURM_JOBID/meld/data/voice/derivatives/meld/ /spin1/users/nielsond/meld/data/voice/derivatives/meld/ ; rm -rf /lscratch/$SLUR

In [330]:
!swarm -f {pwr_swarm_path} --module singularity -g 100 -t 56 --logdir {log_dir} --time 6:00:00 --partition=nimh,norm --gres=lscratch:200

56515024


In [ ]:
#make afni timing file
bids_stims = pd.read_csv(bids_dir/'task-passivelistening_events.tsv', sep = '\t')
stim_names = []
for tt in np.unique(bids_stims.trial_type):
    stim_names.append(tt)
    stim_file = stim_dir/('task-passivelistening_stim-%s.txt'%tt)
    stim_file.write_text(' '.join(list(bids_stims.query('trial_type == @tt').onset.values.astype('str'))))

In [149]:
def make_sing_call(bids_dir, outdir=None, singularity_image = None, subject = None, session= None, task = None):

    lscratch = '/lscratch/$SLURM_JOBID/work'
    bic = '/work/' + bids_dir.parts[-1]
    oic = '/work/out'
    cmd = ' rsync -a ' + bids_dir.absolute().as_posix() + ' ' +lscratch + \
    '; module load afni; 3drefit -TR 2 /lscratch/$SLURM_JOBID/work/self_con/sub-%s'%subject + '/func/*_bold.nii.gz' + /
    '; singularity exec' + \
    ' -B ' + lscratch + ':/work' + \
    ' ' + singularity_image.as_posix()  + \
    ' /run.py ' + bic + ' ' + oic + ' participant'
    if subject is not None:
        cmd += ' --participant_label="' + subject + '"'
    if session is not None:
        cmd += ' --session_label="' + session + '"'
    if task is not None:
        cmd += ' --task_label="' + task + '"'
    cmd += ' --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 0 -dsets {epi_paths}  -align_opts_aea -cost lpc+ZZ -giant_move -tlrc_base MNI152_T1_2009c+tlrc -tlrc_NL_warp -volreg_align_to MIN_OUTLIER -volreg_align_e2a -volreg_tlrc_warp -blur_size 4.0 -bash -regress_reml_exec -regress_motion_per_run -regress_anaticor_fast -regress_censor_motion 0.3'
    cmd += ' -regress_stim_times ' + bic + '/afni_stim/task-passivelistening_stim-nonvocal.txt ' + bic +'/afni_stim/task-passivelistening_stim-vocal.txt ' + ' -regress_stim_labels nonvocal vocal -regress_stim_types IM IM"'
    cmd += '; rsync -a {lscratch}/out/ '.format(lscratch = lscratch) + outdir.absolute().as_posix() + '/ ;'  
    cmd += ' rm -rf /lscratch/$SLURM_JOB_ID/*'
    return cmd

def run_cmd(cmd):
    output = !{cmd}
    return output

In [150]:
df_subjects  = pd.DataFrame({'sub_dir' : [p.as_posix() for p in bids_dir.glob('sub-*')]})
df_subjects['bids_sub'] = df_subjects.sub_dir.str.extract('.*/sub-([0-9]*)',expand =True)
df_subjects

,sub_dir,bids_sub
0,data/voice/sub-217,217
1,data/voice/sub-216,216
2,data/voice/sub-215,215
3,data/voice/sub-214,214
4,data/voice/sub-213,213
5,data/voice/sub-212,212
6,data/voice/sub-211,211
7,data/voice/sub-210,210
8,data/voice/sub-209,209
9,data/voice/sub-208,208


In [151]:
task = 'emotionalregulation'
cmds = []
for sub in df_subjects.bids_sub:
    cmds.append(make_sing_call(bids_dir, outdir, singularity_image = singularity_image, subject = sub, task = task))

cmds[0]

' rsync -a /spin1/users/nielsond/meld/data/voice /lscratch/$SLURM_JOBID/work; module load afni; 3drefit -TR 2 -atrint sform_code 1 -atrint qform_code 1 /lscratch/$SLURM_JOBID/work/self_con/sub-217/func/*_bold.nii.gz; singularity exec -B /lscratch/$SLURM_JOBID/work:/work singularity/afni_proc-2017-12-08-8efc6de0d185.img /run.py /work/voice /work/out participant --participant_label="217" --task_label="emotionalregulation" --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 0 -dsets {epi_paths}  -align_opts_aea -cost lpc+ZZ -giant_move -tlrc_base MNI152_T1_2009c+tlrc -tlrc_NL_warp -volreg_align_to MIN_OUTLIER -volreg_align_e2a -volreg_tlrc_warp -blur_size 4.0 -bash -regress_reml_exec -regress_motion_per_run -regress_anaticor_fast -regress_censor_motion 0.3 -regress_stim_times /work/voice/afni_stim/task-passivelistening_stim-nonvocal.txt /work/voice/afni_stim/task-passivelis

In [46]:

swarm_path.write_text('\n'.join(cmds[2:]))
                      
print(swarm_path.read_text())

 rsync -a /spin1/users/nielsond/meld/data/self_con /lscratch/$SLURM_JOBID/work; singularity exec -B /lscratch/$SLURM_JOBID/work:/work singularity/afni_proc-2017-12-08-8efc6de0d185.img /run.py /work/self_con /work/out participant --participant_label="26" --task_label="emotionalregulation" --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 0 -dsets {epi_paths}  -align_opts_aea -cost lpc+ZZ -giant_move -tlrc_base MNI152_T1_2009c+tlrc -tlrc_NL_warp -volreg_align_to MIN_OUTLIER -volreg_align_e2a -volreg_tlrc_warp -blur_size 4.0 -bash -regress_reml_exec -regress_motion_per_run -regress_anaticor_fast -regress_censor_motion 0.3"; rsync -a /lscratch/$SLURM_JOBID/work/out/ /spin1/users/nielsond/meld/data/self_con/derivatives/afni_proc/ ; rm -rf /lscratch/$SLURM_JOB_ID/*
 rsync -a /spin1/users/nielsond/meld/data/self_con /lscratch/$SLURM_JOBID/work; singularity exec -B /lscratch/$

In [47]:

if not log_dir.exists():
    log_dir.mkdir()

!swarm -f {swarm_path} --module singularity -g 8 -t 4 --logdir {log_dir} --time 48:00:00 --partition=nimh,norm --gres=lscratch:200

55780293


# Voice dataset

In [212]:
bids_dir = Path('data/voice/')
outdir = Path('data/voice//derivatives/afni_proc2')
stim_dir = bids_dir/'afni_stim'
#!rm -rf {outdir}
singularity_image = Path('singularity/afnilocal-2017-12-12-f17aac02375a.img')


if not outdir.exists():
    outdir.mkdir()
if not stim_dir.exists():
    stim_dir.mkdir()

swarm_path_voice = Path('proc_swarm_voice.cmd')
log_dir = Path('swarm_log')

In [133]:
#make afni timing file
bids_stims = pd.read_csv(bids_dir/'task-passivelistening_events.tsv', sep = '\t')
stim_names = []
for tt in np.unique(bids_stims.trial_type):
    stim_names.append(tt)
    stim_file = stim_dir/('task-passivelistening_stim-%s.txt'%tt)
    stim_file.write_text(' '.join(list((bids_stims.query('trial_type == @tt').onset.values - 6).astype('str'))))

In [49]:
df_subjects  = pd.DataFrame({'sub_dir' : [p.as_posix() for p in bids_dir.glob('sub-*')]})
df_subjects['bids_sub'] = df_subjects.sub_dir.str.extract('.*/sub-([0-9]*)',expand =True)
df_subjects

,sub_dir,bids_sub
0,data/voice/sub-217,217
1,data/voice/sub-216,216
2,data/voice/sub-215,215
3,data/voice/sub-214,214
4,data/voice/sub-213,213
5,data/voice/sub-212,212
6,data/voice/sub-211,211
7,data/voice/sub-210,210
8,data/voice/sub-209,209
9,data/voice/sub-208,208


In [221]:
def make_sing_call(bids_dir, outdir=None, singularity_image = None, subject = None, session= None, task = None):

    lscratch = '/lscratch/$SLURM_JOBID/work'
    bic = '/work/' + bids_dir.parts[-1]
    oic = '/work/out'
    cmd = 'rsync -a ' + bids_dir.absolute().as_posix() + ' ' +lscratch + \
    '; module load afni;  gunzip /lscratch/$SLURM_JOBID/work/voice/sub-%s'%subject + '/func/*_bold.nii.gz; 3drefit -TR 2 /lscratch/$SLURM_JOBID/work/voice/sub-%s'%subject + '/func/*_bold.nii' + \
    "; nifti_tool -mod_hdr -overwrite -mod_field sform_code 1 -mod_field qform_code 1 " + "-infiles /lscratch/$SLURM_JOBID/work/voice/sub-%s"%subject + '/func/*_bold.nii' + \
    "; gzip -r /lscratch/$SLURM_JOBID/work/voice/sub-%s"%subject + '/func/*_bold.nii' + \
    "; nifti_tool -disp_hdr -field sform_code -field qform_code -infiles /lscratch/$SLURM_JOBID/work/voice/sub-%s"%subject + '/func/*_bold.nii.gz' + \
    '; export SINGULARITYENV_AFNI_NIFTI_VIEW=orig; singularity exec' + \
    ' -B ' + lscratch + ':/work' + \
    ' ' + singularity_image.as_posix()  + \
    ' /run.py ' + bic + ' ' + oic + ' participant'
    if subject is not None:
        cmd += ' --participant_label="' + subject + '"'
    if session is not None:
        cmd += ' --session_label="' + session + '"'
    if task is not None:
        cmd += ' --task_label="' + task + '"'
    cmd += ' --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 3 -dsets {epi_paths} -tshift_opts_ts -tpattern seq+z -align_opts_aea -cost lpc+ZZ -giant_move -tlrc_base MNI152_T1_2009c+tlrc -tlrc_NL_warp -volreg_align_to MIN_OUTLIER -volreg_align_e2a -volreg_tlrc_warp -blur_size 4.0 -bash -regress_reml_exec -regress_motion_per_run -regress_anaticor_fast -regress_censor_motion 0.3'
    cmd += ' -regress_stim_times ' + bic + '/afni_stim/task-passivelistening_stim-nonvocal.txt ' + bic +'/afni_stim/task-passivelistening_stim-vocal.txt ' + ' -regress_stim_labels nonvocal vocal -regress_stim_types IM IM -regress_basis BLOCK(8,1)"'
    cmd += '; rsync -a {lscratch}/out/ '.format(lscratch = lscratch) + outdir.absolute().as_posix() + '/ ;'  
    cmd += ' rm -rf /lscratch/$SLURM_JOB_ID/*'
    return cmd

def run_cmd(cmd):
    output = !{cmd}
    return output

df_subjects  = pd.DataFrame({'sub_dir' : [p.as_posix() for p in bids_dir.glob('sub-*')]})
df_subjects['bids_sub'] = df_subjects.sub_dir.str.extract('.*/sub-([0-9]*)',expand =True)
df_subjects


,sub_dir,bids_sub
0,data/voice/sub-217,217
1,data/voice/sub-216,216
2,data/voice/sub-215,215
3,data/voice/sub-214,214
4,data/voice/sub-213,213
5,data/voice/sub-212,212
6,data/voice/sub-211,211
7,data/voice/sub-210,210
8,data/voice/sub-209,209
9,data/voice/sub-208,208


In [222]:
cmds = []
for sub in df_subjects.bids_sub:
    cmds.append(make_sing_call(bids_dir, outdir, singularity_image = singularity_image, subject = sub))

cmds[0]

'rsync -a /spin1/users/nielsond/meld/data/voice /lscratch/$SLURM_JOBID/work; module load afni;  gunzip /lscratch/$SLURM_JOBID/work/voice/sub-217/func/*_bold.nii.gz; 3drefit -TR 2 /lscratch/$SLURM_JOBID/work/voice/sub-217/func/*_bold.nii; nifti_tool -mod_hdr -overwrite -mod_field sform_code 1 -mod_field qform_code 1 -infiles /lscratch/$SLURM_JOBID/work/voice/sub-217/func/*_bold.nii; gzip -r /lscratch/$SLURM_JOBID/work/voice/sub-217/func/*_bold.nii; nifti_tool -disp_hdr -field sform_code -field qform_code -infiles /lscratch/$SLURM_JOBID/work/voice/sub-217/func/*_bold.nii.gz; export SINGULARITYENV_AFNI_NIFTI_VIEW=orig; singularity exec -B /lscratch/$SLURM_JOBID/work:/work singularity/afnilocal-2017-12-12-f17aac02375a.img /run.py /work/voice /work/out participant --participant_label="217" --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 3 -dsets {epi_paths} -tshift_opts_t

In [225]:

swarm_path_voice.write_text('\n'.join(cmds[2:]))
                      
print(swarm_path_voice.read_text())

rsync -a /spin1/users/nielsond/meld/data/voice /lscratch/$SLURM_JOBID/work; module load afni;  gunzip /lscratch/$SLURM_JOBID/work/voice/sub-215/func/*_bold.nii.gz; 3drefit -TR 2 /lscratch/$SLURM_JOBID/work/voice/sub-215/func/*_bold.nii; nifti_tool -mod_hdr -overwrite -mod_field sform_code 1 -mod_field qform_code 1 -infiles /lscratch/$SLURM_JOBID/work/voice/sub-215/func/*_bold.nii; gzip -r /lscratch/$SLURM_JOBID/work/voice/sub-215/func/*_bold.nii; nifti_tool -disp_hdr -field sform_code -field qform_code -infiles /lscratch/$SLURM_JOBID/work/voice/sub-215/func/*_bold.nii.gz; export SINGULARITYENV_AFNI_NIFTI_VIEW=orig; singularity exec -B /lscratch/$SLURM_JOBID/work:/work singularity/afnilocal-2017-12-12-f17aac02375a.img /run.py /work/voice /work/out participant --participant_label="215" --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 3 -dsets {epi_paths} -tshift_opts_ts

In [226]:

if not log_dir.exists():
    log_dir.mkdir()

!swarm -f {swarm_path_voice} --module singularity -g 8 -t 4 --logdir {log_dir} --time 12:00:00 --partition=nimh,norm --gres=lscratch:200

56098698


In [220]:
cmds = []
cmds.append(make_sing_call(bids_dir, outdir, singularity_image = singularity_image, subject = '007'))

cmds[0]

swarm_path_voice.write_text('\n'.join(cmds))
                      
print(swarm_path_voice.read_text())

if not log_dir.exists():
    log_dir.mkdir()

!swarm -f {swarm_path_voice} --module singularity -g 8 -t 4 --logdir {log_dir} --time 12:00:00 --partition=nimh,norm --gres=lscratch:200

rsync -a /spin1/users/nielsond/meld/data/voice /lscratch/$SLURM_JOBID/work; module load afni;  gunzip /lscratch/$SLURM_JOBID/work/voice/sub-007/func/*_bold.nii.gz; 3drefit -TR 2 /lscratch/$SLURM_JOBID/work/voice/sub-007/func/*_bold.nii; nifti_tool -mod_hdr -overwrite -mod_field sform_code 1 -mod_field qform_code 1 -infiles /lscratch/$SLURM_JOBID/work/voice/sub-007/func/*_bold.nii; gzip -r /lscratch/$SLURM_JOBID/work/voice/sub-007/func/*_bold.nii; nifti_tool -disp_hdr -field sform_code -field qform_code -infiles /lscratch/$SLURM_JOBID/work/voice/sub-007/func/*_bold.nii.gz; export SINGULARITYENV_AFNI_NIFTI_VIEW=orig; singularity exec -B /lscratch/$SLURM_JOBID/work:/work singularity/afnilocal-2017-12-12-f17aac02375a.img /run.py /work/voice /work/out participant --participant_label="007" --afni_proc="-subj_id {subj_id} -scr_overwrite -out_dir {out_dir} -blocks tshift align tlrc volreg blur mask scale regress -copy_anat {anat_path} -tcat_remove_first_trs 3 -dsets {epi_paths} -tshift_opts_ts